In [41]:
from rouge_score import rouge_scorer
import pandas as pd
pd.set_option("display.max_colwidth", None)
from itertools import product
from statistics import mean

In [44]:
# Pagerank predictions
df = pd.read_pickle("../data/pagerank-test-keypoints.pkl")

In [3]:
df.count()

topic         6
stance        6
ranked_kps    6
gt-kps        6
dtype: int64

In [28]:
generated = df['ranked_kps'].tolist()
references  = df['gt-kps'].tolist()
# take top 10 ranked sentences
top_10_generated = [g[:10] for g in generated]

predictions = []
for item in top_10_generated:
    # get only the sentence and exclude the score
    keypoints = [a for a, b in item]
    predictions.append(keypoints)

In [43]:
rouge1_scores = []
rouge2_scores = []
rougel_scores = []
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
for preds, refs in zip(predictions, references):
    # compute per topic avg scores on all unique combinations of generated keypoints and ground-truth
    r_1 = []
    r_2 = []
    r_l = []
    for a,b in product(preds, refs):
        scores  = scorer.score(a,b)
        r_1.append(round(scores['rouge1'].fmeasure,3))
        r_2.append(round(scores['rouge2'].fmeasure,3))
        r_l.append(round(scores['rougeL'].fmeasure,3))
    
    # save per topic scores to compute average over all topics as the final score
    rouge1_scores.append(round(mean(r_1),3))
    rouge2_scores.append(round(mean(r_2),3))
    rougel_scores.append(round(mean(r_l),3))
print("Rouge 1: {}".format(round(mean(rouge1_scores),3)))
print("Rouge 2: {}".format(round(mean(rouge2_scores),3)))
print("Rouge L: {}".format(round(mean(rougel_scores),3)))

Rouge 1: 0.202
Rouge 2: 0.042
Rouge L: 0.186
